# PNN for Brain Tumor Detection based on DWT

## Import the Dataset and Libraries
In this section we need to get the dataset. The dataset is available on a github repository. The repository contains images of brain tumor and normal brain images. The author has also performed augmentation on the dataset, and it is also provided. We use the augmented images for our training purpose.
<br>Once the repository is cloned, we will import all the required libraries that we need for our problem.

In [ ]:
!git  clone https://github.com/MohamedAliHabib/Brain-Tumor-Detection

Cloning into 'Brain-Tumor-Detection'...
remote: Enumerating objects: 2363, done.
remote: Counting objects: 100% (11/11), done.
remote: Compressing objects: 100% (7/7), done.
remote: Total 2363 (delta 8), reused 4 (delta 4), pack-reused 2352
Receiving objects: 100% (2363/2363), 43.21 MiB | 29.82 MiB/s, done.
Resolving deltas: 100% (22/22), done.


In [ ]:
# Import necessary libraries
import numpy as np
import os
from skimage.feature import greycomatrix, greycoprops
from skimage.color import rgb2gray
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
import matplotlib.pyplot as plt  # Import plt module for visualization
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from skimage.transform import resize
from skimage.feature import graycoprops, graycomatrix
from sklearn.model_selection import train_test_split


## Installing required libraries
We need to install the libraries we are going to use for defining an example PNN for our problem.

In [ ]:
!pip install neupy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.8/226.8 kB 7.2 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of neupy to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.0/226.0 kB 21.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.7/224.7 kB 19.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.3/212.3 kB 18.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 208.5/208.5 kB 20.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 208.0/208.0 kB 18.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 198.6/198.6 kB 17.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 197.2/197.2 kB 19.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 56.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) 

In [ ]:
!pip install neurolab

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 645.3/645.3 kB 17.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for neurolab: filename=neurolab-0.3.5-py3-none-any.whl size=22180 sha256=7cced61e644accc5934e106af0f6f865c3e6b4d06ba079fb9f6b98fe1b9579c4
  Stored in directory: /root/.cache/pip/wheels/1d/c0/44/7142fa43c89473c5e63a750a00224e5f9ec9ca80613de1f97d
Successfully built neurolab


## Declaring Variables

In [ ]:
# Set directory path and parameters
data_dir = '/content/Brain-Tumor-Detection/augmented data'
categories = ['yes', 'no']  # Names of the categories
img_size = 256  # Size of the image after resizing
num_classes = len(categories)
batch_size = 16



## Feature Extraction
Here will define a function named extract_features(). this method takes image path as input and returns the features for each image. Alongwith the feature extraction, the preprocessing on the images is done. <br>
In this method, we calculate the DWT (Discrete Wavelet Transform) features for each image so that the features maybe used for the training purposes. The DWT features are calculated based on mean, variance, maximum, minimum, energy, and entropy. We have calculated six features in this example.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pywt
from skimage.transform import resize
from skimage.color import rgb2gray

def extract_features(image_path):
    # Load image and resize
    img = plt.imread(image_path)
    img_size = 256  # Specify the desired image size
    img = resize(img, (img_size, img_size), anti_aliasing=True)

    # Convert to grayscale
    img_gray = rgb2gray(img)

    # Calculate DWT coefficients
    cA, (cH, cV, cD) = pywt.dwt2(img_gray, 'haar')

    # Extract DWT features
    dwt_mean = np.mean(cA)
    dwt_var = np.var(cA)
    dwt_max = np.max(cA)
    dwt_min = np.min(cA)
    dwt_energy = np.sum(np.square(cA))
    cA_squared = np.square(cA)
    epsilon = 1e-10  # Small constant to avoid division by zero
    dwt_entropy = -np.sum(cA_squared * np.log(cA_squared + epsilon))

    # Return features as an array
    features = np.array([dwt_mean, dwt_var, dwt_max, dwt_min, dwt_energy, dwt_entropy])
    return features


## Extracting Features and Preparing for Training
In this step, we load each image, and calculate the features using the method defined earlier. <br>
Finally, the features are split in train/val categories for training and testing.

In [ ]:
# Load data and extract features
X = []
y = []
for category in categories:
    path = os.path.join(data_dir, category)
    for img_name in os.listdir(path):
        img_path = os.path.join(path, img_name)
        features = extract_features(img_path)
        X.append(features)
        y.append(categories.index(category))

# Convert to numpy arrays
X = np.array(X)
y = np.array(y)

# Split data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

## Model Definition
Here, an example PNN is defined for the explaination purposes. And, this PNN can further be improved for attaining more accuracy.

In [ ]:
# Importing the required library for our PNN definition
import neurolab as nl
# Define the PNN model architecture
input_size = X_train.shape[1]
output_size = 1 # Binary classification task
pnn = nl.net.newp(nl.tool.minmax(X_train), output_size)

## Model Training
In this step, we train the model. We run the training for 10 epochs total. The hyperparameters (batch_size and epochs) can be changed for attaining better results.

In [ ]:
# Train the PNN model
pnn.train(X_train, y_train.reshape(-1, output_size), epochs=50)

The maximum number of train epochs is reached


[390.5,
 338.5,
 355.5,
 435.5,
 353.0,
 386.5,
 335.5,
 435.5,
 406.0,
 332.0,
 332.0,
 331.5,
 301.0,
 418.5,
 308.0,
 363.5,
 406.5,
 335.5,
 357.0,
 333.0,
 373.0,
 331.0,
 435.5,
 405.0,
 338.0,
 405.5,
 385.5,
 332.0,
 435.5,
 349.5,
 363.5,
 307.5,
 361.0,
 383.0,
 376.0,
 353.5,
 386.5,
 406.5,
 299.5,
 309.5,
 435.5,
 307.0,
 440.5,
 323.0,
 435.5,
 370.0,
 332.5,
 355.0,
 435.5,
 356.0]

## Model Testing
Now the accuracy and loss is calculated for our model. Here, val dataset is used, we can devide our dataset into three categories, train, test, and val as well. 

In [ ]:
# Evaluate the PNN model on the test data
predictions = pnn.sim(X_val)
accuracy = (predictions == y_val.reshape(-1, output_size)).mean()
print(f'Test accuracy: {accuracy}')

Test accuracy: 0.5786924939467313
